<a href="https://colab.research.google.com/github/BSmaybe/Almaty-district-clustering-/blob/main/Almaty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
address = 'Almaty'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address) 
latitude_munich = location.latitude
longitude_munich = location.longitude
print('The geograpical coordinate of Almaty are {}, {}.'.format(latitude_munich, longitude_munich))

The geograpical coordinate of Almaty are 43.2363924, 76.9457275.


In [22]:
from google.colab import files
uploaded = files.upload()

Saving data.xlsx to data (1).xlsx


In [23]:
almaty_data_list = pd.read_excel("data.xlsx")
almaty_data = almaty_data_list
almaty_data

,District,postal code
0,Almaly,"050000, 050001, 050002, 050004, 050005, 050008..."
1,Auezov,"050000, 050031, 050035, 050036, 050042, 050043..."
2,Bostandyk,"050000, 050001, 050008, 050012, 050013, 050016..."
3,Medeu,"050000, 050002, 050004, 050010, 050013, 050016..."
4,Turksib,"050001, 050011, 050039, 050054, 050056"
5,Zhetysu,"050004, 050014, 050016, 050018, 050034, 050050"
6,Nauryzbayskiy,050006
7,Alatau,"050014, 050061"


In [29]:
almaty_data_cleaned = pd.DataFrame(columns=['District', 'postal code'])
almaty_data_cleaned.head()

,District,postal code


In [30]:
items = []
for idx, codes in enumerate(almaty_data['postal code']):
    code_list = codes.split(',')
    district = almaty_data['District'][idx]
    for element in code_list:
        element = element.replace(' ', '')
        items.append({'District': district, 'postal Code': element})

In [31]:
almaty_data_cleaned = almaty_data_cleaned.append(items)
almaty_data_cleaned.head()

,District,postal code,postal Code
0,Almaly,NaN,050000
1,Almaly,NaN,050001
2,Almaly,NaN,050002
3,Almaly,NaN,050004
4,Almaly,NaN,050005


In [34]:
almaty_data_cleaned.drop('postal code', inplace=True, axis=1)

In [35]:
almaty_data_cleaned.head()

,District,postal Code
0,Almaly,050000
1,Almaly,050001
2,Almaly,050002
3,Almaly,050004
4,Almaly,050005


In [36]:
# credentials
CLIENT_ID = 'ETS34GMBRT3YRFW4DXYSQDIG4QBMPPCTH1Y0ARYNWGCB41M4' # your Foursquare ID
CLIENT_SECRET = 'VNUSSHWKCDP1PGG3TKMONTMEE3CV30B2AINIG3B1LKE5Y2QG' # your Foursquare Secret
VERSION = '20210912' # Foursquare API version 12/09/2021
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ETS34GMBRT3YRFW4DXYSQDIG4QBMPPCTH1Y0ARYNWGCB41M4
CLIENT_SECRET:VNUSSHWKCDP1PGG3TKMONTMEE3CV30B2AINIG3B1LKE5Y2QG


In [46]:
almaty_data_cleaned.to_csv('file1.csv')

In [67]:
# create new dataframe additionally containing the latitude and longitude values of each district and postal code mapping 
almaty_data_ll = pd.DataFrame(columns=['District', 'Postal Code', 'Latitude', 'Longitude'])

# loop over all entries of old data frame and store according values
items = []
for idx, district in enumerate(almaty_data_cleaned['District']):
    code = almaty_data_cleaned['postal Code'][idx]
    address = district + ', ' + code # to get format of address

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
   # latitude = location.latitude
   # longitude = location.longitude
    items.append({'District': district, 
                  'Postal Code': code,
                  'Latitude': latitude,
                  'Longitude': longitude})

In [70]:
almaty_data_ll = almaty_data_ll.append(items)
almaty_data_ll.head(15)

,District,Postal Code,Latitude,Longitude
0,Almaly,050000,43.294396,76.893719
1,Almaly,050001,43.294396,76.893719
2,Almaly,050002,43.294396,76.893719
3,Almaly,050004,43.294396,76.893719
4,Almaly,050005,43.294396,76.893719
5,Almaly,050008,43.294396,76.893719
6,Almaly,050009,43.294396,76.893719
7,Almaly,050010,43.294396,76.893719
8,Almaly,050012,43.294396,76.893719
9,Almaly,050016,43.294396,76.893719


In [71]:
# create map of munich using latitude and longitude values
map_almaty = folium.Map(location=[almaty_data_ll["Latitude"].iloc[0], almaty_data_ll["Longitude"].iloc[0]], zoom_start=11)

# add markers to map
for lat, lng, district in zip(almaty_data_ll['Latitude'], almaty_data_ll['Longitude'], almaty_data_ll['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_almaty)  
    
map_almaty

In [72]:
# function for getting all venues of munich
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
almaty_venues = getNearbyVenues(names=almaty_data_ll['District'],
                                   latitudes=almaty_data_ll['Latitude'],
                                   longitudes=almaty_data_ll['Longitude']
                                  )

In [ ]:
# set num of clusters
num_clusters = 5
almaty_grouped_clustering = munich_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X)

